In [5]:
# Import necessary libraries
import pandas as pd

## Approach
We want to clean and combine all of our various datasets and export it as one CSV. This will primarily involve dropping columns that we aren't interested in and merging onto a central Pandas DataFrame. After some exploratory analysis, we will determine what rows will beed to be dropped or interpolated.

### County Info
The vaccine hesitancy dataset has values for multiple segments (ethnicity, social vulnerability, vaccine hesitancy), which we will split out into separate variables and look at each. Our primary index of county codes is given by Federal Information Processing Standards (FIPS), which we extract with useful identifiers of the county (its name and state).

In [20]:
vaccine_hesitancy = pd.read_csv('datasets/raw/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv').rename(columns = {'FIPS Code':'fips'})
county = vaccine_hesitancy[['fips', 'County Name', 'State']].rename(columns={'County Name': 'county_name', 'State': 'state'})
county['state'] = county['state'].str.title()
county

fips                     county_name      state
0      1131          Wilcox County, Alabama    Alabama
1      1129      Washington County, Alabama    Alabama
2      1133         Winston County, Alabama    Alabama
3      1127          Walker County, Alabama    Alabama
4      2013  Aleutians East Borough, Alaska     Alaska
...     ...                             ...        ...
3137  55079     Milwaukee County, Wisconsin  Wisconsin
3138  55121   Trempealeau County, Wisconsin  Wisconsin
3139  56001          Albany County, Wyoming    Wyoming
3140  55067      Langlade County, Wisconsin  Wisconsin
3141  55099         Price County, Wisconsin  Wisconsin

[3142 rows x 3 columns]

### Ethnicity 
Percentage of ethnicity for each county are given. For readability and simplicity, we rename them with the most abundant ethnic group as primary and assume non-Hispanic for all of the non-Hispanic groups.

In [7]:
ethnicity = vaccine_hesitancy[['fips', 'Percent Hispanic', 'Percent non-Hispanic American Indian/Alaska Native', 'Percent non-Hispanic Asian', 'Percent non-Hispanic Black', 'Percent non-Hispanic Native Hawaiian/Pacific Islander', 'Percent non-Hispanic White']].rename(columns = {'Percent Hispanic': 'ethnicity_hispanic', 'Percent non-Hispanic American Indian/Alaska Native': 'ethnicity_native', 'Percent non-Hispanic Asian': 'ethnicity_asian', 'Percent non-Hispanic Black': 'ethnicity_black', 'Percent non-Hispanic Native Hawaiian/Pacific Islander': 'ethnicity_hawaiian', 'Percent non-Hispanic White': 'ethnicity_white'})
ethnicity

fips  ethnicity_hispanic  ethnicity_native  ethnicity_asian  \
0      1131              0.0053            0.0009           0.0003   
1      1129              0.0146            0.0731           0.0025   
2      1133              0.0315            0.0034           0.0016   
3      1127              0.0249            0.0015           0.0049   
4      2013              0.0901            0.4588           0.1968   
...     ...                 ...               ...              ...   
3137  55079              0.1500            0.0047           0.0428   
3138  55121              0.0840            0.0034           0.0043   
3139  56001              0.0953            0.0091           0.0327   
3140  55067              0.0197            0.0069           0.0022   
3141  55099              0.0177            0.0096           0.0060   

      ethnicity_black  ethnicity_hawaiian  ethnicity_white  
0              0.6938              0.0000           0.2684  
1              0.2354              0.0000           0.6495  
2              0.0073              0.0005           0.9370  
3              0.0617              0.0000           0.8895  
4              0.0322              0.0100           0.1321  
...               ...                 ...              ...  
3137           0.2606              0.0002           0.5124  
3138           0.0051              0.0000           0.8953  
3139           0.0150              0.0003           0.8248  
3140           0.0125              0.0002           0.9383  
3141           0.0083              0.0085           0.9407  

[3142 rows x 7 columns]

### Social Vulnerability

In [8]:
social_vulnerability_index = vaccine_hesitancy[['fips', 'Social Vulnerability Index (SVI)']].rename(columns= {'Social Vulnerability Index (SVI)': 'social_vulnerability_index'})
social_vulnerability_index

fips  social_vulnerability_index
0      1131                        0.93
1      1129                        0.73
2      1133                        0.70
3      1127                        0.75
4      2013                        0.58
...     ...                         ...
3137  55079                        0.81
3138  55121                        0.28
3139  56001                        0.25
3140  55067                        0.35
3141  55099                        0.16

[3142 rows x 2 columns]

### Vaccine Hesitancy

In [9]:
vaccine_hesitancy = vaccine_hesitancy[['fips', 'Estimated hesitant', 'Estimated strongly hesitant']].rename(columns = {'Estimated hesitant': 'vaccine_hesitant', 'Estimated strongly hesitant': 'vaccine_hesitant_strong'})
vaccine_hesitancy

fips  vaccine_hesitant  vaccine_hesitant_strong
0      1131              0.23                     0.11
1      1129              0.23                     0.11
2      1133              0.22                     0.11
3      1127              0.23                     0.11
4      2013              0.26                     0.12
...     ...               ...                      ...
3137  55079              0.18                     0.11
3138  55121              0.18                     0.10
3139  56001              0.30                     0.16
3140  55067              0.17                     0.10
3141  55099              0.17                     0.10

[3142 rows x 3 columns]

## Education Dataset

We keep four columns from the education dataset representing percentage of the entire adult population with specific educational attainment signifiers and make sure to represent all the percentages as decimal values.

In [31]:
education = pd.read_csv('datasets/raw/Education.csv')
education = education[['FIPS Code', 'Percent of adults with less than a high school diploma, 2015-19', 'Percent of adults with a high school diploma only, 2015-19', "Percent of adults completing some college or associate's degree, 2015-19", "Percent of adults with a bachelor's degree or higher, 2015-19"]]
education = education.rename(columns = {'FIPS Code': 'fips', 'Percent of adults with less than a high school diploma, 2015-19': 'education_high_school_less', 'Percent of adults with a high school diploma only, 2015-19': 'education_high_school_only', "Percent of adults completing some college or associate's degree, 2015-19": 'education_degree_some', "Percent of adults with a bachelor's degree or higher, 2015-19": 'education_bachelors_degree'})
education_cols = ['education_high_school_less', 'education_high_school_only', 'education_degree_some', 'education_bachelors_degree']
education[education_cols] = education[education_cols].div(100)
education

fips  education_high_school_less  education_high_school_only  \
0         0                       0.120                       0.270   
1      1000                       0.138                       0.308   
2      1001                       0.115                       0.336   
3      1003                       0.092                       0.277   
4      1005                       0.268                       0.356   
...     ...                         ...                         ...   
3278  72145                       0.284                       0.262   
3279  72147                       0.288                       0.392   
3280  72149                       0.220                       0.384   
3281  72151                       0.290                       0.257   
3282  72153                       0.266                       0.333   

      education_degree_some  education_bachelors_degree  
0                     0.289                       0.321  
1                     0.299                       0.255  
2                     0.284                       0.266  
3                     0.313                       0.319  
4                     0.260                       0.116  
...                     ...                         ...  
3278                  0.241                       0.212  
3279                  0.140                       0.180  
3280                  0.197                       0.199  
3281                  0.272                       0.180  
3282                  0.155                       0.246  

[3283 rows x 5 columns]

Number of missing county entries in our education dataset:  0


### Finding missing data
Now that we've extracted out the useful data from that dataset, we check if we're missing any data from our master vaccine hesitancy dataset of county info. Using pandas  `isin` as a boolean mask on the two county index keys, we do what is more or less a difference between the two sets. Luckily, all our education data is complete and nothing is missing.

In [46]:
# Show missing county rows in education dataset
county[~county.fips.isin(education.fips)]

Empty DataFrame
Columns: [fips, county_name, state]
Index: []

### Poverty
From the Poverty Estimate dataset, we keep the column of values for the percentage of the entire population that is in poverty in 2019. The data is in a narrow format, so we use pivot and make sure to represent the percentage as a decimal.

In [38]:
poverty = pd.read_csv('datasets/raw/PovertyEstimates.csv')
poverty = poverty[['FIPStxt', 'Attribute', 'Value']].pivot(index='FIPStxt', columns='Attribute', values='Value').reset_index()
poverty = poverty[['FIPStxt', 'PCTPOVALL_2019']].rename(columns = {'FIPStxt':'fips', 'PCTPOVALL_2019': 'poverty'})
poverty['poverty'] = poverty['poverty'].div(100)
display(poverty)

Attribute   fips  poverty
0              0    0.123
1           1000    0.156
2           1001    0.121
3           1003    0.101
4           1005    0.271
...          ...      ...
3188       56037    0.083
3189       56039    0.060
3190       56041    0.085
3191       56043    0.111
3192       56045    0.105

[3193 rows x 2 columns]

In [37]:
# Show missing county rows in poverty dataset
county[~county.fips.isin(poverty.fips)]

fips             county_name   state
48  15005  Kalawao County, Hawaii  Hawaii

## Natality

In [12]:
natality = pd.read_csv('datasets/raw/PopulationEstimates.csv')
natality = natality[['FIPStxt', 'POP_ESTIMATE_2019', 'R_birth_2019']].rename(columns = {'FIPStxt': 'fips', 'POP_ESTIMATE_2019': 'population', 'R_birth_2019': 'birth_rate'})
natality['birth_rate'] = natality['birth_rate'].div(100)
natality

fips   population  birth_rate
0         0  328,239,523         NaN
1      1000    4,903,185       0.117
2      1001       55,869       0.112
3      1003      223,234       0.104
4      1005       24,686       0.103
...     ...          ...         ...
3268  72145       50,023         NaN
3269  72147        8,386         NaN
3270  72149       21,372         NaN
3271  72151       32,282         NaN
3272  72153       33,575         NaN

[3273 rows x 3 columns]

In [39]:
# Show missing county rows in natality dataset
county[~county.fips.isin(natality.fips)]

Empty DataFrame
Columns: [fips, county_name, state]
Index: []

In [13]:
election_years = [2008, 2012, 2016, 2020]
def election_winner(row, year):
    if row['dem_' + str(year)] > row['gop_' + str(year)]:
        return 'Democrat'
    else:
        return 'Republican'
elections_data = pd.read_csv('datasets/raw/US_County_Level_Presidential_Results_08-16.csv').rename(columns={'fips_code': 'fips'})
elections_data_2020 = pd.read_csv('datasets/raw/2020_US_County_Level_Presidential_Results.csv').rename(columns={'votes_gop': 'gop_2020', 'votes_dem': 'dem_2020', 'county_fips': 'fips'})
elections_data = elections_data.merge(elections_data_2020[['fips', 'gop_2020', 'dem_2020']], left_on='fips', right_on='fips')
elections = pd.DataFrame()
elections['fips'] = elections_data['fips']
for year in election_years:
    elections['election_' + str(year)] = elections_data.apply(lambda row: election_winner(row, year), axis=1)
elections

fips election_2008 election_2012 election_2016 election_2020
0     26041      Democrat    Republican    Republican    Republican
1     48295    Republican    Republican    Republican    Republican
2      1127    Republican    Republican    Republican    Republican
3     48389      Democrat      Democrat      Democrat    Republican
4     56017    Republican    Republican    Republican    Republican
...     ...           ...           ...           ...           ...
3106  17115      Democrat    Republican    Republican    Republican
3107  29215    Republican    Republican    Republican    Republican
3108  46051    Republican    Republican    Republican    Republican
3109  17103    Republican    Republican    Republican    Republican
3110  20063    Republican    Republican    Republican    Republican

[3111 rows x 5 columns]

In [40]:
# Show missing county rows in elections dataset
county[~county.fips.isin(elections.fips)]

fips                                county_name         state
4      2013             Aleutians East Borough, Alaska        Alaska
5      2016         Aleutians West Census Area, Alaska        Alaska
24     2050                 Bethel Census Area, Alaska        Alaska
48    15005                     Kalawao County, Hawaii        Hawaii
371    2020             Anchorage Municipality, Alaska        Alaska
375    2198  Prince of Wales-Hyder Census Area, Alaska        Alaska
392    2275          Wrangell City and Borough, Alaska        Alaska
394    2122            Kenai Peninsula Borough, Alaska        Alaska
398    2180                   Nome Census Area, Alaska        Alaska
402    2261         Valdez-Cordova Census Area, Alaska        Alaska
403    2220             Sitka City and Borough, Alaska        Alaska
407    2090       Fairbanks North Star Borough, Alaska        Alaska
411    2185                North Slope Borough, Alaska        Alaska
424    2110            Juneau City and Borough, Alaska        Alaska
430    2070             Dillingham Census Area, Alaska        Alaska
446    2150              Kodiak Island Borough, Alaska        Alaska
449    2060                Bristol Bay Borough, Alaska        Alaska
454    2188           Northwest Arctic Borough, Alaska        Alaska
456    2158               Kusilvak Census Area, Alaska        Alaska
457    2105          Hoonah-Angoon Census Area, Alaska        Alaska
463    2240    Southeast Fairbanks Census Area, Alaska        Alaska
470    2290          Yukon-Koyukuk Census Area, Alaska        Alaska
476    2068                     Denali Borough, Alaska        Alaska
480    2195                 Petersburg Borough, Alaska        Alaska
489    2100                     Haines Borough, Alaska        Alaska
492    2164         Lake and Peninsula Borough, Alaska        Alaska
502    2130          Ketchikan Gateway Borough, Alaska        Alaska
503    2282           Yakutat City and Borough, Alaska        Alaska
507    2230               Skagway Municipality, Alaska        Alaska
510    2170          Matanuska-Susitna Borough, Alaska        Alaska
2724  46102         Oglala Lakota County, South Dakota  South Dakota

## Unemployment Dataset
From the Unemployment dataset, we have several useful data points involving geography (rural vs urban continuum code, urban influence code), income (median household, and represented as a percent of median state total) and unemployment rate.

In [14]:
unemployment = pd.read_csv('datasets/raw/Unemployment.csv').pivot(index='fips_txt', columns='Attribute', values='Value').reset_index().rename(columns = {'fips_txt':'fips'})
geography = unemployment[['fips', 'Rural_urban_continuum_code_2013', 'Urban_influence_code_2013']].rename(columns={'Rural_urban_continuum_code_2013': 'rural_urban_code', 'Urban_influence_code_2013': 'urban_influence_code'})
# TODO: convert urban/rural codes into z-scores
geography

Attribute   fips  rural_urban_code  urban_influence_code
0              0               NaN                   NaN
1           1000               NaN                   NaN
2           1001               2.0                   2.0
3           1003               3.0                   2.0
4           1005               6.0                   6.0
...          ...               ...                   ...
3270       72145               1.0                   1.0
3271       72147               7.0                  12.0
3272       72149               2.0                   2.0
3273       72151               1.0                   1.0
3274       72153               2.0                   2.0

[3275 rows x 3 columns]

In [41]:
# Show missing county rows in geography dataset
county[~county.fips.isin(geography.fips)]

fips             county_name   state
48  15005  Kalawao County, Hawaii  Hawaii

### Income
To look at the county's economic factors, we keep two columns representing the estimated median household income in 2019 and the county household median income as a percent of the state total median household income. We represent this percent as a decimal.

In [15]:
income = unemployment[['fips', 'Med_HH_Income_Percent_of_State_Total_2019', 'Median_Household_Income_2019']].rename(columns={'Med_HH_Income_Percent_of_State_Total_2019': 'median_income_percent_state', 'Median_Household_Income_2019': 'median_income'})
income['median_income_percent_state'] = income['median_income_percent_state'].div(100)
income

Attribute   fips  median_income_percent_state  median_income
0              0                          NaN        65712.0
1           1000                     1.000000        51771.0
2           1001                     1.124819        58233.0
3           1003                     1.156458        59871.0
4           1005                     0.694829        35972.0
...          ...                          ...            ...
3270       72145                          NaN            NaN
3271       72147                          NaN            NaN
3272       72149                          NaN            NaN
3273       72151                          NaN            NaN
3274       72153                          NaN            NaN

[3275 rows x 3 columns]

In [42]:
# Show missing county rows in income dataset
county[~county.fips.isin(income.fips)]

fips             county_name   state
48  15005  Kalawao County, Hawaii  Hawaii

In [16]:
unemployment = unemployment[['fips', 'Unemployment_rate_2019']].rename(columns={'Unemployment_rate_2019': 'unemployment'})
unemployment['unemployment'] = unemployment['unemployment'].div(100)
unemployment

Attribute   fips  unemployment
0              0      0.036694
1           1000      0.030000
2           1001      0.027000
3           1003      0.027000
4           1005      0.038000
...          ...           ...
3270       72145      0.096000
3271       72147      0.069000
3272       72149      0.159000
3273       72151      0.131000
3274       72153      0.146000

[3275 rows x 2 columns]

In [43]:
# Show missing county rows in unemployment dataset
county[~county.fips.isin(unemployment.fips)]

fips             county_name   state
48  15005  Kalawao County, Hawaii  Hawaii

In [17]:
religion = pd.read_csv('datasets/raw/U.S. Religion Census Religious Congregations and Membership Study, 2010 (County File).csv')
# print(religion.columns.tolist())
religion = religion[['FIPS', 'TOTRATE', 'EVANRATE', 'BPRTRATE', 'MPRTRATE', 'CATHRATE', 'ORTHRATE', 'OTHRATE']].rename(columns = {'FIPS': 'fips', 'TOTRATE': 'religion_total', 'EVANRATE': 'religion_evangelical', 'BPRTRATE': 'religion_black_protestant', 'MPRTRATE': 'religion_mainline_protestant', 'CATHRATE': 'religion_catholic', 'ORTHRATE': 'religion_orthodox', 'OTHRATE': 'religion_other'})
religion
# TODO: decide to drop black_protestant and/or orthodox columns

fips  religion_total  religion_evangelical  religion_black_protestant  \
0      1001      676.878889            503.990000                  41.978889   
1      1003      531.740000            318.138889                  17.170000   
2      1005      549.990000            320.250000                 121.208889   
3      1007      498.800000            443.328889                  42.158889   
4      1009      651.620000            509.800000                   1.010000   
...     ...             ...                   ...                        ...   
3144  56037      477.220000             99.120000                        NaN   
3145  56039      260.360000             42.030000                        NaN   
3146  56041      606.830000             68.658889                        NaN   
3147  56043      471.818889            155.870000                        NaN   
3148  56045      280.518889             81.578889                        NaN   

      religion_mainline_protestant  religion_catholic  religion_orthodox  \
0                        82.858889          32.358889                NaN   
1                       110.140000          76.858889               1.04   
2                        77.938889          20.940000                NaN   
3                        13.178889                NaN                NaN   
4                        52.950000          82.760000                NaN   
...                            ...                ...                ...   
3144                     31.230000         159.960000               4.45   
3145                     85.468889          65.608889                NaN   
3146                     23.488889          38.020000                NaN   
3147                     88.708889          94.810000                NaN   
3148                     70.060000          78.658889                NaN   

      religion_other  
0          15.688889  
1           8.380000  
2           9.650000  
3           0.130000  
4           5.088889  
...              ...  
3144      182.460000  
3145       67.250000  
3146      476.648889  
3147      132.430000  
3148       50.218889  

[3149 rows x 8 columns]

In [44]:
# Show missing county rows in religion dataset
county[~county.fips.isin(religion.fips)]

fips                         county_name         state
456    2158        Kusilvak Census Area, Alaska        Alaska
2724  46102  Oglala Lakota County, South Dakota  South Dakota

## Data Completeness
As noted above we are missing a few recurring pattern of certain counties being missing in most of our datasets.
* Oglala Lakota County, South Dakota
* Kalawao County, Hawaii
* Various parts of Alaska, especially in the election dataset

Upon further investigation into those areas, we learned that Oglala Lakota County does not have a functioning county seat and remains unorganized, which explains the difficulty government surveyers would have with gathering data there. However, this county is entirely on an Indian reservation, which would give us valuable insight on Native American vaccine hesitancy.

Kalawao County because of its small population does not have many of the functions that a normal county would have.

While Alaska does administer using county divisions, for elections they use a different geographic boundary of boroughs, which do not conveniently align with counties. This makes a county level political correlation with vaccine hesitancy impossible for us.

Because of the issues surrounding population size, county organization and governmental issues, we decide to drop these data points.

In [ ]:
# TODO: drop data points

## Aggregation
We set the fips as index for all of our dataframes and then concatenate them along it with an inner join. We note that there's only one row that was lost.

In [45]:
dfs = [df.set_index('fips') for df in [county, vaccine_hesitancy, social_vulnerability_index, ethnicity, natality, unemployment, geography, income, poverty, education]]
# TODO: add politics, natality after data cleaned
df = pd.concat(dfs, axis=1, join='inner').reset_index()
df

fips                     county_name      state  \
0      1131          Wilcox County, Alabama    Alabama   
1      1129      Washington County, Alabama    Alabama   
2      1133         Winston County, Alabama    Alabama   
3      1127          Walker County, Alabama    Alabama   
4      2013  Aleutians East Borough, Alaska     Alaska   
...     ...                             ...        ...   
3136  55079     Milwaukee County, Wisconsin  Wisconsin   
3137  55121   Trempealeau County, Wisconsin  Wisconsin   
3138  56001          Albany County, Wyoming    Wyoming   
3139  55067      Langlade County, Wisconsin  Wisconsin   
3140  55099         Price County, Wisconsin  Wisconsin   

                         County Name      State  Estimated hesitant  \
0             Wilcox County, Alabama    ALABAMA                0.23   
1         Washington County, Alabama    ALABAMA                0.23   
2            Winston County, Alabama    ALABAMA                0.22   
3             Walker County, Alabama    ALABAMA                0.23   
4     Aleutians East Borough, Alaska     ALASKA                0.26   
...                              ...        ...                 ...   
3136     Milwaukee County, Wisconsin  WISCONSIN                0.18   
3137   Trempealeau County, Wisconsin  WISCONSIN                0.18   
3138          Albany County, Wyoming    WYOMING                0.30   
3139      Langlade County, Wisconsin  WISCONSIN                0.17   
3140         Price County, Wisconsin  WISCONSIN                0.17   

      Estimated strongly hesitant  Social Vulnerability Index (SVI)  \
0                            0.11                              0.93   
1                            0.11                              0.73   
2                            0.11                              0.70   
3                            0.11                              0.75   
4                            0.12                              0.58   
...                           ...                               ...   
3136                         0.11                              0.81   
3137                         0.10                              0.28   
3138                         0.16                              0.25   
3139                         0.10                              0.35   
3140                         0.10                              0.16   

           SVI Category  CVAC level of concern for vaccination rollout  ...  \
0     Very High Concern                                           0.94  ...   
1          High Concern                                           0.82  ...   
2          High Concern                                           0.80  ...   
3          High Concern                                           0.68  ...   
4      Moderate Concern                                           0.87  ...   
...                 ...                                            ...  ...   
3136  Very High Concern                                           0.30  ...   
3137        Low Concern                                           0.31  ...   
3138        Low Concern                                           0.63  ...   
3139        Low Concern                                           0.19  ...   
3140   Very Low Concern                                           0.20  ...   

     unemployment  rural_urban_code  urban_influence_code  \
0           0.071               9.0                  10.0   
1           0.046               8.0                   7.0   
2           0.033               6.0                   4.0   
3           0.033               1.0                   1.0   
4           0.028               9.0                  12.0   
...           ...               ...                   ...   
3136        0.040               1.0                   1.0   
3137        0.036               6.0                   6.0   
3138        0.031               4.0                   5.0   
3139        0.042               6.0          

## Output

In [57]:
df.to_csv('datasets/clean/interim_clean_dataset_2021-05-21.csv')